# Using skforecast models

This is an example for using skforecast based models with the `timecopilot` library.

## Imports

In [1]:
import nest_asyncio

nest_asyncio.apply()

from timecopilot import TimeCopilot
from timecopilot.models.adapters import SKForecastAdapter


from skforecast.recursive import  ForecasterRecursiveMultiSeries
from skforecast.preprocessing import RollingFeatures

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingRegressor

import pandas as pd

## Setup the SKForecast model

skforecast models can be passed in the `forecasters` argument when initializing the TimeCopilot agent where they will be wrapped in an adapter with an alias based on the type name. 

If multiple skforecast forecasters of the same type are passed, each model after the first will have be wrapped in an adapter with an alias that has `'_n'` appended to it with `n` being incremented by 1 for each additional occurrence of the same model type. 

For example, if you pass two `ForecasterRecursive` skforecast models, the first one will have an alias of `'skforecast.ForecasterRecursive'` and the second one will have an alias of `'skforecast.ForecasterRecursive_2'`. 

If you would rather specify the alias yourself, you will need to adapt the model manually with `SKForecastAdapter`.

Note: when using a single series forecasting model, each series will be forecasted separately. You can check if a forecaster is a single series model by checking if `get_tags()['forecasting_scope']` returns `'single-series'` or skforecast's documentation.

In [2]:
window_features = RollingFeatures(stats=['mean', 'min', 'max'], window_sizes=7)
skf_forecaster = ForecasterRecursiveMultiSeries(
    estimator          = HistGradientBoostingRegressor(random_state=8523),
    lags               = 10,
    encoding           = 'ordinal',
    transformer_series = StandardScaler(),
    window_features    = window_features,
)

### Manually adapt skforecast model

If you would rather decide on the alias yourself, you will need to manually adapt the model with `SKForecastAdapter`.

The `model` argument should be an skforecast `Forecaster` model. The `alias` argument should be a string that uniquely identifies the model.

After adapting the model you would pass it in the `forecasters` argument when initializing the TimeCopilot agent.

If you add multiple manually adapted skforecast models of the same type without specifying aliases, TimeCopilot may not be able to properly call all of them.

In [ ]:
manually_adapted_model = SKForecastAdapter(
    model=skf_forecaster,
    alias="TrendForecaster",
)

tc = TimeCopilot(
    llm="openai:gpt-4o",
    forecasters=[
        manually_adapted_model
    ]
)

## Create a TimeCopilot instance with your sktime model

You will need to specify the forecasters you're using when using sktime models. 

In [3]:
tc = TimeCopilot(
    llm="openai:gpt-4o",
    forecasters=[
        skf_forecaster,
    ],
)

### Extending default model list with an skforecast model

if you want to use the default list with the addition of your skforecast model you could make a copy of the default list and append your model to it:

In [ ]:
from timecopilot.agent import DEFAULT_MODELS

model_list = DEFAULT_MODELS.copy()
model_list.append(skf_forecaster)

tc = TimeCopilot(llm="openai:gpt-4o", forecasters=model_list)

## Forecasting 
Once that setup is complete, you can use TimeCopilot with your adapted skforecast model the same way you'd normally use TimeCopilot

In [4]:
df = pd.read_csv("https://timecopilot.s3.amazonaws.com/public/data/air_passengers.csv")

In [5]:
result = tc.forecast(
    df=df,
)

1it [00:00, 30.56it/s]
1it [00:00, 218.25it/s]
11it [00:00, 267.69it/s]


In [6]:
print(result.output.tsfeatures_analysis)

The AirPassengers time series exhibits strong seasonal patterns with a seasonality period of 12 months, as indicated by the strong seasonal strength of 0.981. The trend component is quite robust and nearly linear (unitroot_kpss of 2.739 suggests trending behavior while a unitroot_pp of -6.566 indicates it's stationary in its seasonal differences). The high autocorrelation in the level series (x_acf1 of 0.948) directly reflects strong autoregressive tendencies.


In [7]:
display(result.fcst_df)

,unique_id,ds,SeasonalNaive
0,AirPassengers,1961-01-01,417.0
1,AirPassengers,1961-02-01,391.0
2,AirPassengers,1961-03-01,419.0
3,AirPassengers,1961-04-01,461.0
4,AirPassengers,1961-05-01,472.0
5,AirPassengers,1961-06-01,535.0
6,AirPassengers,1961-07-01,622.0
7,AirPassengers,1961-08-01,606.0
8,AirPassengers,1961-09-01,508.0
9,AirPassengers,1961-10-01,461.0
